# Dans ce Mini-projet, nous allons créer un Robot-chat relativement simple qui sera utilisé pour répondre aux questions fréquemment posées.

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json

with open('intents.json') as file:
    data = json.load(file)

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
    words = []
    labels = []
    docs_words_tok = []
    docs_tag = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_words_tok.append(wrds)
            docs_tag.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

In [3]:
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

In [4]:
    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for idx, doc in enumerate(docs_words_tok):
        bag = []
        
        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_tag[idx])] = 1

        training.append(bag)
        output.append(output_row)
   

    training = numpy.array(training)
    output = numpy.array(output)

In [5]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

modele = tflearn.DNN(net)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [6]:
modele.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
modele.save("modele.tflearn")

Training Step: 3999  | total loss: 0.58033 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.58033 - acc: 0.9291 -- iter: 24/26
Training Step: 4000  | total loss: 0.53145 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.53145 - acc: 0.9362 -- iter: 26/26
--
INFO:tensorflow:C:\Users\my-pc\Desktop\TAL-NLP-Project-master\fileMourad\modele.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
#La fonction bag_of_words transformera notre entrée de chaîne en un sac de mots en utilisant notre liste de mots créée

def bag_of_words(sentence, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(sentence)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for sentc in s_words:
        for indx, word in enumerate(words):
            if word == sentc:
                bag[indx] = 1
            
    return numpy.array(bag)

In [ ]:
#La fonction de chat gérera l'obtention d'une prédiction du modèle et la saisie d'une réponse 
# appropriée à partir de notre fichier de réponses JSON
def chat():
    print("Commencer la conversation avec le Robot (Tapez stop pour arrêter la concersation)!")
    while True:
        entree = input("You: ")
        if entree.lower() == "stop":
            break

        resultat = modele.predict([bag_of_words(entree, words)])
        resultat_index = numpy.argmax(resultat)
        tag = labels[resultat_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                reponses = tg['responses']

        print(random.choice(reponses))
#chat()
chat()

Commencer la conversation avec le Robot (Tapez stop pour arrêter la concersation)!
